In [1]:
import pmagpy
import pmagpy.pmagplotlib as pmagplotlib
import pmagpy.ipmag as ipmag
import pmagpy.pmag as pmag
import pmagpy.contribution_builder as cb
from pmagpy import convert_2_magic as convert
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import glob as glob
import shutil
import math
from pathlib import Path
import time
from tqdm import tqdm
%matplotlib inline

In [2]:
home = Path(os.getcwd())
data_folder = home/'data'
magic_folder = home/'magic'

In [76]:
def add_vgps(sites_df):
    # Outputs dataframe with columns: vgp_lat, vgp_lon, vgp_dp, vgp_dm
    data=sites_df[['dir_dec','dir_inc','dir_alpha95','lat','lon']].to_numpy()
    vgps=np.array(pmag.dia_vgp(data)) 
    vgps=vgps.transpose()
    return(pd.DataFrame(vgps,columns=['vgp_lon','vgp_lat','vgp_dp','vgp_dm'],index=sites_df.index))

def dir_df_boot_ci(dir_df, nb=500, column_map=None, return_distribution=False):
    """
    Performs a bootstrap for direction DataFrame with parametric bootstrap,
    providing bootstrap kappa parameter

    Parameters
    ----------
    dir_df : Pandas DataFrame with columns:
        dir_dec : mean declination
        dir_inc : mean inclination
        dir_n : number of data points in mean
        dir_k : Fisher k statistic for mean
    nb : number of bootstraps, default is 5000
    return_distribution : return DataFrame of dec, inc, kappa values
            (default: False)

    Returns
    -------
    if return_distribution is False:
        boot_results: Pandas DataFrame with columns:
            dir_dec : bootstrapped median declination value
            dir_inc : bootstrapped median inclination value
            dir_k : bootstrapped median kappa value
            dir_n : number of rows in original dataframe
            dir_alpha95: A95 associated with medioan kappa
    if return_distribution is True:
        boot_results: Pandas DataFrame with all bootstrapped values of columns above
    
  
    """
    dir_df=dir_df.dropna()
    N = dir_df.shape[0] # Note: N counts only non-NaN elements
    if N>1:
        if column_map is not None:
            dir_df=dir_df.rename(columns=column_map)
        all_boot_results = pd.DataFrame(np.zeros((nb,5)),columns=['dir_dec','dir_inc','dir_n','dir_k','dir_alpha95'])
        t0=time.clock()
        for k in tqdm(range(nb)):
            boot_di=dir_df.apply(lambda x: np.array(list(ipmag.fishrot(k=x['dir_k'],n=int(x['dir_n']),dec=x['dir_dec'],inc=x['dir_inc'],di_block=False))).T,axis=1).explode().apply(lambda x: pd.Series({'dir_dec':x[0],'dir_inc':x[1]}))
            all_boot_results.iloc[k,:] = pd.Series(pmag.dir_df_fisher_mean(boot_di)).drop(index=['csd','r']).rename(index={'alpha95':'dir_alpha95',
                                      'dec':'dir_dec',
                                      'inc':'dir_inc',
                                      'k':'dir_k',
                                      'n':'dir_n'})
        t1=time.clock()
        t_total=t1-t0

        b = 20.**(1./(N -1.))-1.
        if return_distribution:
            results = all_boot_results
            results['a']=1-b*(N-1.)/((N*(results['dir_k']-1.))-1.)
            results['dir_n']=N
            results['dir_alpha95']=np.degrees(np.arccos(results['a']))
            results['result_type']='a'
        else:
            fisher_avg=pd.DataFrame([pmag.dir_df_fisher_mean(all_boot_results)]).rename(columns={'alpha95':'dir_alpha95',
                                      'dec':'dir_dec',
                                      'inc':'dir_inc',
                                      'k':'dir_k',
                                      'n':'dir_n'})
            fisher_avg['dir_k'] = np.median(all_boot_results['dir_k'].to_numpy())
            fisher_avg['dir_n'] = N
            a=1-b*(N-1.)/((N*(fisher_avg['dir_k']-1.))-1.)
            fisher_avg['dir_alpha95'] = np.degrees(np.arccos(a))
            fisher_avg['time']=t_total
            results=fisher_avg[['dir_dec','dir_inc','dir_k','dir_n','dir_alpha95','time']]
            results['result_type']='a'
    else:
        results=dir_df[['dir_dec','dir_inc','dir_k','dir_alpha95']]
        results['dir_n']=1
        results['time']=0
        results['result_type']='i'
    if column_map is not None:
        column_map_r=dict(zip(column_map.values(),column_map.keys()))
        results=results.rename(columns=column_map_r)
    return(results)

## Letts et al. 2009

In [4]:
## Letts et al (2009)
#%%capture sites_conversion_log
# Capture output to sites_conversion_log

## Convert Sites XLSX Files to MagIC format

# In this XLSX file, here is how I interpret the mapping of relevant columns to the MagIC 3.0 data model (PAS 8/16/2024): 
#  
#
#  Region: sites.location (Note: there are also 'locations' for each )
#  DecLong: sites.lon
#  DecLat: sites.lat
#  n (col. L): sites.dir_n_total_samples (Note: as per conversation with Jeff and Rachel 10/15/2024, this may not be correct!)
#  Full Site: sites.site (had to add this column to remove ambiguities in site numbers)
#  GDec: sites.dir_dec
#  GInc: sites.dir_inc
#  Strike: sites.bed_dip_direction (=(strike+90.)%360.)
#  Dip: sites.bed_dip_direction
#  n (col. W): sites.dir_n_samples (Note: as per conversation with Jeff and Rachel 10/15/2024, this may not be correct!)
#  a95 (col. X): sites.dir_alpha95
#  kappa: sites.dir_k
#  Lithology: sites.lithologies
#  
# Assumptions:
#
#  sites.formation = "Bushveld Complex"
#  sites.geologic_classes = "Igenous:Intrusive"
#  sites.geologic_types = "Layered Inntrusion"
#  sites.result_type = "i"
#  sites.method_codes = "DE-BFL:DE-K:LP-DIR-AF:LP-DIR-T"
#  sites.citations = "10.1111/j.1365-246X.2009.04346.x"
#  sites.age = 2054
#  sites.age_unit = "Ma"
#  sites.dir_tilt_correction = 0 (geographc coordinates)
#  sites.result_quality = "g" if n (col. W) is not nan, otherwise "b"

letts_data = pd.read_excel(data_folder/'Letts_Bushveld.xlsx',sheet_name='Sheet1')
#display(letts_data)

In [5]:
# Compile sites dataframe
sites_df=pd.DataFrame([])
sites_df['site']=letts_data['Full Site'].astype('string')
#sites_df['location']=letts_data.apply(lambda x: "{}:{}".format(x['Region'],x['Site']),axis=1)
sites_df['location']=letts_data['Region']
sites_df['lat']=letts_data['DecLat']
sites_df['lon']=letts_data['DecLong']
sites_df['dir_n_total_samples']=letts_data['n']
sites_df['dir_n_samples']=letts_data['n.1']
sites_df['dir_dec']=letts_data['GDec']
sites_df['dir_inc']=letts_data['GInc']
sites_df['bed_dip']=letts_data['Dip']
sites_df['bed_dip_direction']=(letts_data['Strike']+90.)%360.
sites_df['dir_alpha95']=letts_data['a95.1']
sites_df['dir_k']=letts_data['kappa']
sites_df['lithologies']= letts_data['Lithology']
sites_df['formation']= "Bushveld Complex"
sites_df['geologic_classes']='Igneous:Intrusive'
sites_df['result_type']= "i"
sites_df['method_codes']= "DE-BFL:DE-K:LP-DIR-AF:LP-DIR-T"
sites_df['citations']= "10.1111/j.1365-246X.2009.04346.x"
sites_df['geologic_types']="Layered Intrusion"
sites_df['age']= 2054
sites_df['age_unit']= "Ma"
sites_df['dir_tilt_correction']=0
sites_df['result_quality']= letts_data.apply(lambda x: 'b' if np.isnan(x['n.1']) else 'g',axis=1)
#display(sites_df)

In [6]:
# Calculate and attach VGPs
# NOTE: Needs to be updated so that VGPs are calculated based on tilt-corrected data!
vgps_df=add_vgps(sites_df)
sites_df = pd.concat([sites_df,vgps_df],axis=1)

### Interlude: How many bootstraps do we need to determine kappa when averaging site-level data?

_Note: Here we test the bootstrap to determine a "total" kappa value when averaging site means. We tested two groups of sites from Letts, rows 0-2 and 6-11. In both cases, the nb=500 bootstrap both provided a low enough variance (standard deviation of kappa from 7 runs is about 1% of the mean) and enough precision (average kappa of 7 runs is within 5% of the value at nb=10000) in a short enough time to make the bootstrap feasible._

**SKIP THIS SECTION (following two blocks of code) UNLESS YOU REED TO RETEST THE BOOTSTRAP. This takes a really long time to run.**

In [ ]:
to_avg=sites_df[sites_df['location']=='Main Zone Western Lobe'][['dir_dec','dir_inc','dir_n_samples','dir_k']].iloc[0:3,:]
print(to_avg)
print('Fisher Mean:')
print(pd.DataFrame(pmag.dir_df_fisher_mean(to_avg),index=[0]))
to_avg.columns=['dir_dec','dir_inc','dir_n','dir_k']
boots=[10, 20, 50, 50, 50, 50, 50, 50, 50, 100, 200, 300, 400, 500, 600, 700, 500, 500, 500, 500, 500, 500,
       500, 800, 900, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 10000]
results=pd.DataFrame([],columns=['dir_dec','dir_inc','dir_n','dir_k','dir_alpha95'])
for nb in boots:
    print(f'Bootstrap (nb = {nb}):')
    result=dir_df_boot_ci(to_avg,nb=nb)
    result['nb']=nb
    results=results.append(result)
print(results)

In [ ]:
result_averages = results.groupby('nb').apply(np.mean)
result_std = results.groupby('nb').apply(np.std)
print(result_std)
big_kappa=result_averages.loc[10000,'dir_k']
print(big_kappa)
results['q']=100.*np.abs(results['dir_k']-big_kappa)/big_kappa
print(result_averages)
ax=plt.subplot(111)
ax.plot(boots,100*np.abs(results['dir_k']-big_kappa)/big_kappa,'ko')
ax2 = ax.twinx()
ax2.plot(boots,results['dir_alpha95'],'bo')
plt.title('Scree Plot')
ax.set_xlabel('Number of bootstrap realizations')
ax.set_ylabel('$100\\frac{\kappa_i - \\kappa_{10000}}{\\kappa_{10000}}$')
ax2.set_ylabel('$\\alpha_{95}$',color='blue')
# Note: 500 is probably enough. Gives about 5% difference from 1000 average, std dev is ~1%

_Back to our regularly-scheduled programming. If you are running this notebook to get MagIC data files, this is where the Letts (2009) processing resumes after testing the bootstraps._

In [7]:
# In this block and the next, we create a "locations.txt" file by averaging sites for two different definitions of "location": 
#  Averaging by the "location" column (i.e. Main Zone Eastern Lobe, etc.)
#  Averaging over the entire complex
# These are initially created as two dataframes that are then merged in the next block.

# Averaging by location:
# Mapping names of columns. We will average site-level data. 
# Each site has a number of samples listed in the sites table, so we will use that as our N in the parametric bootstrap.
sample_map={'dir_n_samples':'dir_n'} 
# Compile locations dataframe
locations_df1=pd.DataFrame([])
sites_group=sites_df.groupby('location')
locations_df1['location']=sites_group.groups.keys()
locations_df1['sites']=sites_group['site'].unique().apply(':'.join).reset_index(drop=True) # What are all the sites in this location?
locations_df1['location_type']='Region'
locations_df1['geologic_classes']='Igneous:Intrusive'
locations_df1['lithologies']=sites_group['lithologies'].unique().apply(':'.join).reset_index(drop=True) # What are all the lithologies in this location?
# The following 4 lines find the bounding box of coordinates for each grouping of sites. 
locations_df1['lat_s']=sites_group['lat'].min().reset_index(drop=True) 
locations_df1['lat_n']=sites_group['lat'].max().reset_index(drop=True)
locations_df1['lon_e']=sites_group['lon'].max().reset_index(drop=True)
locations_df1['lon_w']=sites_group['lon'].min().reset_index(drop=True)
locations_df1['age']=2054
locations_df1['age_unit']="Ma"
# The following does a parametric bootstrap for each location to find the average kappa and a95 values
locations_df1=pd.concat([locations_df1,sites_group.apply(lambda x: dir_df_boot_ci(x,column_map=sample_map)).reset_index()],axis=1)
# Need to rename columns: now our "n" is numbers of sites, since locations_df is the locations dataframe (a site-level average).
locations_df1=locations_df1.rename({'dir_n_samples':'dir_n_sites'})

# Averaging for the whole complex:
locations_df2=pd.DataFrame([])
sites_group=sites_df.groupby('formation')
locations_df2['location']=sites_group.groups.keys()
locations_df2['sites']=sites_group['site'].unique().apply(':'.join).reset_index(drop=True)#.drop('formation',axis=1)
locations_df2['location_type']='Region'
locations_df2['geologic_classes']='Igneous:Intrusive'
locations_df2['lithologies']=sites_group['lithologies'].unique().apply(':'.join).reset_index(drop=True)#.drop('formation',axis=1)
locations_df2['lat_s']=sites_group['lat'].min().reset_index(drop=True)
locations_df2['lat_n']=sites_group['lat'].max().reset_index(drop=True)
locations_df2['lon_e']=sites_group['lon'].max().reset_index(drop=True)
locations_df2['lon_w']=sites_group['lon'].min().reset_index(drop=True)
locations_df2['age']=2054
locations_df2['age_unit']="Ma"
locations_df2=pd.concat([locations_df2,sites_group.apply(lambda x: dir_df_boot_ci(x,column_map=sample_map)).reset_index()],axis=1)
locations_df2=locations_df2.rename(columns={'dir_n_samples':'dir_n_sites'})

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [06:55<00:00,  1.20it/s]


In [8]:
# Put the two dataframes (with different definitions of what a "location" is)
locations_df = pd.concat([locations_df1.iloc[:, [j for j, c in enumerate(locations_df1.columns) if j not in [11, 12, 18]]], 
                          locations_df2.iloc[:, [j for j, c in enumerate(locations_df2.columns) if j not in [11, 12, 18]]]],
                         axis=0,ignore_index=True)
display(locations_df)

,location,sites,location_type,geologic_classes,lithologies,lat_s,lat_n,lon_e,lon_w,age,age_unit,dir_dec,dir_inc,dir_k,dir_n_samples,dir_alpha95,result_type
0,Critical Zone,CZ-4:CZ-66:CZ-67:CZ-68:CZ-M:CZ-B1:CZ-B2:CZ-H2:...,Region,Igneous:Intrusive,Norite:Intrusives,-24.886833,-24.401167,30.138667,27.310700,2054,Ma,182.800827,-66.125362,2.410020,8,50.483510,a
1,Main Zone Eastern Lobe,MZE-1:MZE-2a:MZE-2b:MZE-3:MZE-8:MZE-42:MZE-43:...,Region,Igneous:Intrusive,Gabbronorite:Norite,-25.689000,-24.335333,30.044333,29.379500,2054,Ma,31.062137,61.384889,1.948814,26,30.055835,a
2,Main Zone Northern Lobe,MZN-1:MZN-2:MZN-3:MZN-4:MZN-5:MZN-6:MZN-7:MZN-...,Region,Igneous:Intrusive,Intrusives,-24.227333,-23.433333,29.034500,28.117500,2054,Ma,33.720439,55.540433,7.402014,22,12.319019,a
3,Main Zone Western Lobe,MZW-10:MZW-11:MZW-12:MZW-13:MZW-14:MZW-15:MZW-...,Region,Igneous:Intrusive,Gabbronorite,-25.627000,-25.514333,28.214833,27.307333,2054,Ma,352.744731,78.957455,12.141909,31,7.755715,a
4,Upper Zone,UZ-5:UZ-9:UZ-39:UZ-40:UZ-57:UZ-60:UZ-64:UZ-70,Region,Igneous:Intrusive,Gabbro:Gabbronorite:Pyroxenite,-25.381500,-24.415333,29.896333,27.207167,2054,Ma,169.601257,-53.461650,2.056818,8,60.103076,a
5,Bushveld Complex,MZW-10:MZW-11:MZW-12:MZW-13:MZW-14:MZW-15:MZW-...,Region,Igneous:Intrusive,Gabbronorite:Norite:Intrusives:Gabbro:Pyroxenite,-25.689000,-23.433333,30.138667,27.207167,2054,Ma,30.669313,71.094528,1.995097,95,14.657438,a


In [9]:
# Compile contributions dataframe
# Note that MagIC somehow does not recognize this when you upload it, so you have to specify the DOI and lab by hand anyway.
contribution_df=pd.DataFrame({'reference':["10.1111/j.1365-246X.2009.04346.x"],'lab_names':["Not Specified"]},index=[0])
print(contribution_df)

                          reference      lab_names
0  10.1111/j.1365-246X.2009.04346.x  Not Specified


In [10]:
# Changing dataframes to dictionaries for export
contribution_dicts=contribution_df.fillna('').to_dict('records')
locations_dicts=locations_df.fillna('').to_dict('records')
sites_dicts = sites_df.fillna('').to_dict('records')

In [11]:
# Write files for uploading to MagIC
# You will still need to upload these to a private contribution by hand
pmag.magic_write(magic_folder/'letts2009_contribution.txt', contribution_dicts, 'contribution')
pmag.magic_write(str(magic_folder/'letts2009_locations.txt'), locations_dicts, 'locations')
pmag.magic_write(magic_folder/'letts2009_sites.txt', sites_dicts, 'sites')

1  records written to file  C:\Users\paselkin\Dropbox\Research\intrusion_magnetics\published-data-to-magic\magic\letts2009_contribution.txt
6  records written to file  C:\Users\paselkin\Dropbox\Research\intrusion_magnetics\published-data-to-magic\magic\letts2009_locations.txt
101  records written to file  C:\Users\paselkin\Dropbox\Research\intrusion_magnetics\published-data-to-magic\magic\letts2009_sites.txt


(True,
 WindowsPath('C:/Users/paselkin/Dropbox/Research/intrusion_magnetics/published-data-to-magic/magic/letts2009_sites.txt'))

## Kosterov & Perrin (1996)

In [12]:
## Kosterov & Perrin (1996)
#%%capture sites_conversion_log
# Capture output to sites_conversion_log

## Convert Sites XLSX Files to MagIC format

# In this XLSX file, here is how I interpret the mapping of relevant columns to the MagIC 3.0 data model (PAS 10/07/2024): 
#  
#
#  Location: samples.site
#  ID: samples.sample (had to add this column to remove ambiguities in site numbers)
#  Lat: samples.lat
#  Long: samples.lon
#  N: samples.dir_n_total_specimens
#  n: samples.dir_n_specimens
#  Inc: samples.dir_inc
#  Dec: samples.dir_dec
#  k: samples.dir_k
#  a95: samples.dir_alpha95
#  GroupID: samples.site ()
#  
# Assumptions:
#
#  samples.result_type = "i"
#  samples.method_codes = "DE-BFL:DE-K:LP-DIR-AF:LP-DIR-T"
#  samples.citations = "10.1016/0012-821X(96)00005-2"
#  samples.age = 180
#  samples.age_sigma = 5
#  samples.age_unit = "Ma"
#  samples.dir_tilt_correction = 0
#  samples.result_quality = "g" if group (col. N) is not nan, otherwise "b"

kosterov1996_data = pd.read_excel(data_folder/'KarooData.xlsx',sheet_name='Kosterov 1996',skiprows=4).dropna(how='all')
#display(kosterov1996_data)

In [13]:
# Compile samples dataframe
samples_df=pd.DataFrame([])
samples_df['sample']=kosterov1996_data['ID'].astype('string')
samples_df['site']=kosterov1996_data['GroupID'].astype('string')
samples_df['site']=samples_df['site'].fillna('N/A')
samples_df['location']=kosterov1996_data['Location']
samples_df['lat']=kosterov1996_data['Lat']
samples_df['lon']=kosterov1996_data['Long']
samples_df['dir_n_total_specimens']=kosterov1996_data['N']
samples_df['dir_n_specimens']=kosterov1996_data['n']
samples_df['dir_dec']=kosterov1996_data['Dec']
samples_df['dir_inc']=kosterov1996_data['Inc']
samples_df['dir_alpha95']=kosterov1996_data['a95']
samples_df['dir_k']=kosterov1996_data['k']
samples_df['lithologies']= 'Basalt'
samples_df['geologic_classes']='Igneous:Extrusive'
samples_df['result_type']= "i"
samples_df['method_codes']= "DE-BFL:DE-K:LP-DIR-AF:LP-DIR-T"
samples_df['citations']= "10.1016/0012-821X(96)00005-2"
samples_df['geologic_types']="Lava Flow"
samples_df['age']= 180
samples_df['age_sigma']= 5
samples_df['age_unit']= "Ma"
samples_df['dir_tilt_correction']=0
samples_df['result_quality']= 'g'
samples_df.loc[samples_df['location'].isna(),'result_quality']='b'
with pd.option_context('display.max_rows', 1000, 'display.max_columns', 10):
    display(samples_df)

,sample,site,location,lat,lon,...,age,age_sigma,age_unit,dir_tilt_correction,result_quality
0,M-101,GM-28,Mafika,-29.07,28.38,...,180,5,Ma,0,g
1,M-100,GM-28,Mafika,-29.07,28.38,...,180,5,Ma,0,g
2,M-95,GM-27,Mafika,-29.07,28.38,...,180,5,Ma,0,g
3,M-93,GM-26,Mafika,-29.07,28.38,...,180,5,Ma,0,g
4,M-91,GM-26,Mafika,-29.07,28.38,...,180,5,Ma,0,g
5,M-89,GM-26,Mafika,-29.07,28.38,...,180,5,Ma,0,g
6,M-88,GM-26,Mafika,-29.07,28.38,...,180,5,Ma,0,g
7,M-86,GM-25,Mafika,-29.07,28.38,...,180,5,Ma,0,g
8,M-84,GM-24,Mafika,-29.07,28.38,...,180,5,Ma,0,g
9,M-82,GM-24,Mafika,-29.07,28.38,...,180,5,Ma,0,g


In [14]:
# Calculate and attach VGPs
vgps_df=add_vgps(samples_df)
samples_df = pd.concat([samples_df,vgps_df],axis=1)

In [86]:
sample_map={'dir_n_specimens':'dir_n'}
# Compile sites dataframe
sites_df=pd.DataFrame([])
samples_group=samples_df.groupby('site')
sites_df['site']=samples_group.groups.keys()
sites_df['samples']=samples_group['sample'].unique().apply(':'.join).reset_index(drop=True)#.drop('site',axis=1)
sites_df['site_type']='Outcrop'
sites_df['geologic_classes']=samples_group['geologic_classes'].unique().apply(':'.join).reset_index(drop=True)#.drop('site',axis=1)
sites_df['lithologies']='Basalt'
sites_df['location']=samples_group.first()['location'].reset_index(drop=True)
sites_df['lat_s']=samples_group['lat'].min().reset_index(drop=True)
sites_df['lat_n']=samples_group['lat'].max().reset_index(drop=True)
sites_df['lon_e']=samples_group['lon'].max().reset_index(drop=True)
sites_df['lon_w']=samples_group['lon'].min().reset_index(drop=True)
sites_df['age']=180
sites_df['age_sigma']=5
sites_df['age_unit']="Ma"
sites_df=pd.concat([sites_df,samples_group.apply(lambda x: dir_df_boot_ci(x,column_map=sample_map,nb=500)).reset_index()],axis=1)
sites_df=sites_df.rename(columns={'dir_n_specimens':'dir_n_samples'})

100%|███████████████████████████████████████████████████████████████████████████████| 500/500 [00:04<00:00, 104.39it/s]


In [88]:
location_map={'dir_n_samples':'dir_n'}
# Compile locations dataframe
locations_df=pd.DataFrame([])
sites_group=sites_df.groupby('location')
locations_df['location']=sites_group.groups.keys()
locations_df['sites']=sites_group['location'].unique().apply(':'.join).reset_index(drop=True)
locations_df['location_type']='Outcrop'
locations_df['geologic_classes']=sites_group['geologic_classes'].unique().apply(':'.join).reset_index(drop=True)#.drop('location',axis=1)
locations_df['lithologies']='Basalt'
#locations_df['description']=sites_group.first()['description'].reset_index(drop=True)
locations_df['lat_s']=sites_group['lat_s'].min().reset_index(drop=True)
locations_df['lat_n']=sites_group['lat_n'].max().reset_index(drop=True)
locations_df['lon_e']=sites_group['lon_e'].max().reset_index(drop=True)
locations_df['lon_w']=sites_group['lon_w'].min().reset_index(drop=True)
locations_df['age']=180
locations_df['age_sigma']=5
locations_df['age_unit']="Ma"
locations_df=pd.concat([locations_df,sites_group.apply(lambda x: dir_df_boot_ci(x,column_map=location_map)).reset_index()],axis=1)
locations_df=locations_df.rename(columns={'dir_n_samples':'dir_n_sites'})
with pd.option_context('display.max_rows', 1000, 'display.max_columns', 10):
    display(locations_df)

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:06<00:00, 75.47it/s]


,location,sites,location_type,geologic_classes,lithologies,...,dir_k,dir_n_sites,dir_alpha95,time,result_type
0,Mafika,Mafika,Outcrop,Igneous:Extrusive,Basalt,...,60.493397,28,3.535354,16.132525,a
1,Nazareth,Nazareth,Outcrop,Igneous:Extrusive,Basalt,...,43.644926,4,14.147296,3.463200,a
2,Rhodes,Rhodes,Outcrop,Igneous:Extrusive,Basalt,...,88.613136,3,13.225071,3.261751,a
3,Sani Pass,Sani Pass,Outcrop,Igneous:Extrusive,Basalt,...,34.859448,12,7.473794,6.628269,a


In [89]:
locations_dicts=locations_df.fillna('').to_dict('records')
sites_dicts = sites_df.fillna('').to_dict('records')

In [90]:
pmag.magic_write(str(magic_folder/'kosterov1996_locations.txt'), locations_dicts, 'locations')
pmag.magic_write(magic_folder/'kosterov1996_sites.txt', sites_dicts, 'sites')

4  records written to file  C:\Users\paselkin\Dropbox\Research\intrusion_magnetics\published-data-to-magic\magic\kosterov1996_locations.txt
48  records written to file  C:\Users\paselkin\Dropbox\Research\intrusion_magnetics\published-data-to-magic\magic\kosterov1996_sites.txt


(True,
 WindowsPath('C:/Users/paselkin/Dropbox/Research/intrusion_magnetics/published-data-to-magic/magic/kosterov1996_sites.txt'))

## Moulin et al. 2011

In [ ]:
## Moulin et al. 2011
#%%capture sites_conversion_log
# Capture output to sites_conversion_log

## Convert Sites XLSX Files to MagIC format

# In this XLSX file, here is how I interpret the mapping of relevant columns to the MagIC 3.0 data model (PAS 8/17/2024): 
#  
#
#  Section: sites.description
#  Site: sites.site (had to add this column to remove ambiguities in site numbers)
#  Slat: sites.lat
#  Slon: sites.lon
#  N: sites.dir_n_total_samples
#  n: sites.dir_n_samples
#  Ig: sites.dir_inc
#  Dg: sites.dir_dec
#  k: sites.dir_k
#  a95: sites.dir_alpha95
#  Dir. Group or Flow: sites.location
#  Age: sites.age
#  dAge: sites.age_sigma
#  
# Assumptions:
#
#  sites.result_type = "i"
#  sites.method_codes = "DE-BFL:DE-K:LP-DIR-AF:LP-DIR-T"
#  sites.citations = "10.1029/2011JB008210"
#  sites.age_unit = "Ma"
#  sites.dir_tilt_correction = 0
#  sites.result_quality = "g" if group is not nan, otherwise "b"

moulin2011_data = pd.read_excel(data_folder/'KarooData.xlsx',sheet_name='Moulin 2011',skiprows=4,usecols="A:S").dropna(how='all')
#display(moulin2011_data)

In [ ]:
# Compile sites dataframe
sites_df=pd.DataFrame([])
sites_df['site']=moulin2011_data['Site'].astype('string')
sites_df['formation']=moulin2011_data['Formation'].astype('string')
sites_df['location']=moulin2011_data['Dir. Group or Flow'].astype('string')
sites_df['description']=moulin2011_data['Section']
sites_df['lat']=moulin2011_data['Slat']
sites_df['lon']=moulin2011_data['Slon']
sites_df['dir_n_total_samples']=moulin2011_data['N']
sites_df['dir_n_samples']=moulin2011_data['n']
sites_df['dir_dec']=moulin2011_data['Dg']
sites_df['dir_inc']=moulin2011_data['Ig']
sites_df['dir_alpha95']=moulin2011_data['a95']
sites_df['dir_k']=moulin2011_data['k']
sites_df['lithologies']= 'Basalt'
sites_df['geologic_classes']='Igneous:Extrusive'
sites_df['result_type']= "i"
sites_df['method_codes']= "DE-BFL:DE-K:LP-DIR-AF:LP-DIR-T"
sites_df['citations']= "10.1029/2011JB008210"
sites_df['geologic_types']="Lava Flow"
sites_df['age']= moulin2011_data['Age']
sites_df['age_sigma']= moulin2011_data['dAge']
sites_df['age_unit']= "Ma"
sites_df['dir_tilt_correction']=0
sites_df['result_quality']= 'g'
sites_df.loc[sites_df['location'].isna(),'result_quality']='b'
with pd.option_context('display.max_rows', 1000, 'display.max_columns', 10):
    display(sites_df)

In [ ]:
# Calculate and attach VGPs
vgps_df=add_vgps(sites_df)
sites_df = pd.concat([sites_df,vgps_df],axis=1)

In [ ]:
# Compile locations dataframe
locations_df1=pd.DataFrame([])
sites_group=sites_df.groupby('location')
locations_df1['location']=sites_group.groups.keys()
locations_df1['sites']=sites_group['site'].unique().apply(':'.join).reset_index().drop('location',axis=1)
locations_df1['description']=sites_group.first()['description'].reset_index(drop=True)
locations_df1['location_type']='Outcrop'
locations_df1['geologic_classes']=sites_group['geologic_classes'].unique().apply(':'.join).reset_index().drop('location',axis=1)
locations_df1['lithologies']='Basalt'
locations_df1['lat_s']=sites_df['lat'].min()
locations_df1['lat_n']=sites_df['lat'].max()
locations_df1['lon_e']=sites_df['lon'].max()
locations_df1['lon_w']=sites_df['lon'].min()
locations_df1['age_low']=sites_df['age'].min()
locations_df1['age_high']=sites_df['age'].max()
locations_df1['age_sigma']=sites_df['age_sigma'].max()
locations_df1['age_unit']="Ma"
locations_df1=pd.concat([locations_df1,sites_group.apply(lambda x: fisher_avg(x)).reset_index()[['dir_dec','dir_inc','dir_n_sites','dir_k','dir_alpha95','result_type']]],
                        axis=1)

locations_df2=pd.DataFrame([])
#sites_group=sites_df.groupby('description')
sites_group=locations_df1.groupby('description')

locations_df2['location']=sites_group.groups.keys()
new_sites=sites_group.apply(lambda x:":".join(list(set(x['sites'].str.split(':').explode().tolist())))).reset_index()
new_sites.columns=['location','sites']

locations_df2=pd.merge(locations_df2,new_sites,on='location')
locations_df2['location_type']='Stratigraphic Section'
locations_df2['geologic_classes']=sites_group['geologic_classes'].unique().apply(':'.join).reset_index().drop('description',axis=1)
locations_df2['lithologies']='Basalt'
locations_df2['lat_s']=sites_df['lat'].min()
locations_df2['lat_n']=sites_df['lat'].max()
locations_df2['lon_e']=sites_df['lon'].max()
locations_df2['lon_w']=sites_df['lon'].min()
locations_df2['age_low']=sites_df['age'].min()
locations_df2['age_high']=sites_df['age'].max()
locations_df2['age_sigma']=sites_df['age_sigma'].max()
locations_df2['age_unit']="Ma"
locations_df2=pd.concat([locations_df2,sites_group.apply(lambda x: fisher_avg(x)).reset_index()[['dir_dec','dir_inc','dir_n_sites','dir_k','dir_alpha95','result_type']]],
                        axis=1)

locations_df = pd.concat([locations_df1, locations_df2],axis=0).reset_index(drop=True)
with pd.option_context('display.max_rows', 1000, 'display.max_columns', 10):
    display(locations_df)

In [ ]:
locations_dicts=locations_df.fillna('').to_dict('records')
sites_dicts = sites_df.fillna('').to_dict('records')

In [ ]:
pmag.magic_write(str(magic_folder/'moulin2011_locations.txt'), locations_dicts, 'locations')
pmag.magic_write(magic_folder/'moulin2011_sites.txt', sites_dicts, 'sites')

## Moulin et al. 2012

In [ ]:
## Moulin et al. 2012
#%%capture sites_conversion_log
# Capture output to sites_conversion_log

## Convert Sites XLSX Files to MagIC format

# In this XLSX file, here is how I interpret the mapping of relevant columns to the MagIC 3.0 data model (PAS 8/17/2024): 
#  
#
#  Section: sites.location
#  Site: sites.site (had to add this column to remove ambiguities in site numbers)
#  Lat: sites.lat
#  Long: sites.lon
#  N: sites.dir_n_total_samples
#  n: sites.dir_n_samples
#  Inc: sites.dir_inc
#  Dec: sites.dir_dec
#  k: sites.dir_k
#  a95: sites.dir_alpha95
#  Dir. Group: sites.group
#  
# Assumptions:
#
#  sites.result_type = "i"
#  sites.method_codes = "DE-BFL:DE-K:LP-DIR-AF:LP-DIR-T"
#  sites.citations = "10.1029/2011GC003910"
#  sites.age = 179.2
#  sites.age_sigma = 1.8
#  sites.age_unit = "Ma"
#  sites.dir_tilt_correction = 0
#  sites.result_quality = "g" if group is not nan, otherwise "b"

moulin2012_data = pd.read_excel(data_folder/'KarooData.xlsx',sheet_name='Moulin 2012',skiprows=3,usecols="A:L").dropna(how='all')
display(moulin2012_data)

In [ ]:
# Compile sites dataframe
sites_df=pd.DataFrame([])
sites_df['site']=moulin2012_data['Site'].astype('string')
sites_df['groups']=moulin2012_data['Dir. Group'].astype('string')
sites_df['location']=moulin2012_data['Section']
sites_df['lat']=moulin2012_data['Lat']
sites_df['lon']=moulin2012_data['Long']
sites_df['dir_n_total_samples']=moulin2012_data['N']
sites_df['dir_n_samples']=moulin2012_data['n']
sites_df['dir_dec']=moulin2012_data['Dec']
sites_df['dir_inc']=moulin2012_data['Inc']
sites_df['dir_alpha95']=moulin2012_data['a95']
sites_df['dir_k']=moulin2012_data['k']
sites_df['lithologies']= 'Basalt'
sites_df['geologic_classes']='Igneous:Extrusive'
sites_df['result_type']= "i"
sites_df['method_codes']= "DE-BFL:DE-K:LP-DIR-AF:LP-DIR-T"
sites_df['citations']= "10.1029/2011GC003910"
sites_df['geologic_types']="Lava Flow"
sites_df['age']= 179.2
sites_df['age_sigma']= 1.8
sites_df['age_unit']= "Ma"
sites_df['dir_tilt_correction']=0
sites_df['result_quality']= 'g'
sites_df.loc[sites_df['groups'].isna(),'result_quality']='b'
with pd.option_context('display.max_rows', 1000, 'display.max_columns', 10):
    display(sites_df)

In [ ]:
# Calculate and attach VGPs
vgps_df=add_vgps(sites_df)
sites_df = pd.concat([sites_df,vgps_df],axis=1)

In [ ]:
# Compile locations dataframe
locations_df=pd.DataFrame([])
sites_group=sites_df.groupby('location')
locations_df['location']=sites_group.groups.keys()
locations_df['sites']=sites_group['site'].unique().apply(':'.join).reset_index().drop('location',axis=1)
locations_df['location_type']='Stratigraphic Section'
locations_df['geologic_classes']=sites_group['geologic_classes'].unique().apply(':'.join).reset_index().drop('location',axis=1)
locations_df['lithologies']='Basalt'
locations_df['lat_s']=sites_df['lat'].min()
locations_df['lat_n']=sites_df['lat'].max()
locations_df['lon_e']=sites_df['lon'].max()
locations_df['lon_w']=sites_df['lon'].min()
locations_df['age']=179.2
locations_df['age_sigma']=1.8
locations_df['age_unit']="Ma"
with pd.option_context('display.max_rows', 1000, 'display.max_columns', 10):
    display(locations_df)

In [ ]:
locations_dicts=locations_df.to_dict('records')
sites_dicts = sites_df.to_dict('records')

In [ ]:
pmag.magic_write(str(magic_folder/'moulin2012_locations.txt'), locations_dicts, 'locations')
pmag.magic_write(magic_folder/'moulin2012_sites.txt', sites_dicts, 'sites')

## Moulin et al. 2017

In [ ]:
## Moulin et al. 2017
#%%capture sites_conversion_log
# Capture output to sites_conversion_log

## Convert Sites XLSX Files to MagIC format

# In this XLSX file, here is how I interpret the mapping of relevant columns to the MagIC 3.0 data model (PAS 8/17/2024): 
#  
#
#  Section: sites.location
#  Site: sites.site (had to add this column to remove ambiguities in site numbers)
#  Slat (deg): sites.lat
#  Slon (deg): sites.lon
#  N: sites.dir_n_total_samples
#  n: sites.dir_n_samples
#  Ig (deg): sites.dir_inc
#  Dg (deg): sites.dir_dec
#  K: sites.dir_k
#  a95 (deg): sites.dir_alpha95
#  Directional Group or Single Flow: sites.group
#  
# Assumptions:
#
#  sites.result_type = "i"
#  sites.method_codes = "DE-BFL:DE-K:LP-DIR-AF:LP-DIR-T"
#  sites.citations = "10.1002/2016JB013354"
#  sites.age = 181.1
#  sites.age_sigma = 1.0
#  sites.age_unit = "Ma"
#  sites.dir_tilt_correction = 0
#  sites.result_quality = "g" if group is not nan, otherwise "b"

moulin2017_data = pd.read_excel(data_folder/'KarooData.xlsx',sheet_name='Moulin 2017',skiprows=1,usecols="A:L").dropna(how='all')
display(moulin2017_data)

In [ ]:
# Compile sites dataframe
sites_df=pd.DataFrame([])
sites_df['site']=moulin2017_data['Site'].astype('string')
sites_df['groups']=moulin2017_data['Directional Group or Single Flow'].astype('string')
sites_df['location']=moulin2017_data['Section']
sites_df['lat']=moulin2017_data['Slat (deg)']
sites_df['lon']=moulin2017_data['Slon (deg)']
sites_df['dir_n_total_samples']=moulin2017_data['N']
sites_df['dir_n_samples']=moulin2017_data['n']
sites_df['dir_dec']=moulin2017_data['Dg (deg)']
sites_df['dir_inc']=moulin2017_data['Ig (deg)']
sites_df['dir_alpha95']=moulin2017_data['a95 (deg)']
sites_df['dir_k']=moulin2017_data['K']
sites_df['lithologies']= 'Basalt'
sites_df['geologic_classes']='Igneous:Extrusive'
sites_df['result_type']= "i"
sites_df['method_codes']= "DE-BFL:DE-K:LP-DIR-AF:LP-DIR-T"
sites_df['citations']= "10.1002/2016JB013354"
sites_df['geologic_types']="Lava Flow"
sites_df['age']= 181.1
sites_df['age_sigma']= 1.0
sites_df['age_unit']= "Ma"
sites_df['dir_tilt_correction']=0
sites_df['result_quality']= 'g'
sites_df.loc[sites_df['groups'].isna(),'result_quality']='b'
with pd.option_context('display.max_rows', 1000, 'display.max_columns', 10):
    display(sites_df)

In [ ]:
# Calculate and attach VGPs
vgps_df=add_vgps(sites_df)
sites_df = pd.concat([sites_df,vgps_df],axis=1)

In [ ]:
# Compile locations dataframe
locations_df=pd.DataFrame([])
sites_group=sites_df.groupby('location')
locations_df['location']=sites_group.groups.keys()
locations_df['sites']=sites_group['site'].unique().apply(':'.join).reset_index().drop('location',axis=1)
locations_df['location_type']='Stratigraphic Section'
locations_df['geologic_classes']=sites_group['geologic_classes'].unique().apply(':'.join).reset_index().drop('location',axis=1)
locations_df['lithologies']='Basalt'
locations_df['lat_s']=sites_df['lat'].min()
locations_df['lat_n']=sites_df['lat'].max()
locations_df['lon_e']=sites_df['lon'].max()
locations_df['lon_w']=sites_df['lon'].min()
locations_df['age']=181.1
locations_df['age_sigma']=1.0
locations_df['age_unit']="Ma"
with pd.option_context('display.max_rows', 1000, 'display.max_columns', 10):
    display(locations_df)

In [ ]:
locations_dicts=locations_df.to_dict('records')
sites_dicts = sites_df.to_dict('records')

In [ ]:
pmag.magic_write(str(magic_folder/'moulin2017_locations.txt'), locations_dicts, 'locations')
pmag.magic_write(magic_folder/'moulin2017_sites.txt', sites_dicts, 'sites')